# Is This Lungtumor Cancerous?

This is a Plan B work in supporting Paln A (won't be presented if Plan A is successful).
A dataset from https://www.kaggle.com/datasets/andrewmvd/lung-and-colon-cancer-histopathological-images was selected to train a regular CNN model.
A patient interface was created at the end to tell weather a input lungtumor image indicates a cancer/carcinoma   

Part 0: Importing Dependencies

In [16]:
# Cell(C) 1
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from PIL import Image
import pandas as pd
import requests
import numpy as np

Part 1: Importing Data

In [23]:
# C2
# Dennis Code Cell for loading in dataset lung_image_sets_22Aug2024 (reduced to 300 images)  to Google Colab


import zipfile
import os

#zip_file_path = '/content/sample_data/lung_image_sets_22Aug2024.zip'
#zip_file_path = '/content/100lung_n.zip'
zip_file_path = '/usr/lung_image_sets_22Aug2024.zip'
#extract_folder = '/content/sample_data/lung_image_sets_22Aug2024'
#extract_folder = '/content/100lung_n'
extract_folder = '/usr/lung_image_sets_22Aug2024'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of zip file in the current directory
    zip_ref.extractall(extract_folder)

# Check if the extraction was successful
if os.path.isdir(extract_folder):
    print("Extraction successful.")
else:
    print("Extraction failed.")
#After running this code, you should be able to access the contents of the lung_image_sets_new directory for further processing.

Extraction successful.


In [24]:
# C3

# ##### From Oliver 1

# Load Images:
# Path to the directory containing the images
#image_directory = 'lung_image_sets/lung_aca'
#image_directory = '/content/lung_aca'  #lung_aca1-1000.jpeg
image_directory = extract_folder
                #'/usr/lung_image_sets_22Aug2024/lung_image_sets_22Aug2024/lung_aca/lungaca1-100.jpeg
                #'/usr/lung_image_sets_22Aug2024/lung_image_sets_22Aug2024/lung_n/lungn1-100.jpeg
                #'/usr/lung_image_sets_22Aug2024/lung_image_sets_22Aug2024/lung_n/lungscc1-100.jpeg



# List to store the loaded images
images = []

# Loop through all files in the directory
for filename in os.listdir(image_directory):
    if filename.endswith(".jpeg"):
        # Construct the full path to the image file
        image_path = os.path.join(image_directory, filename)

        # Open the image and append it to the list
        img = Image.open(image_path)
        images.append(img)

print(f"Loaded {len(images)} images.")

Loaded 0 images.


Part 2: preprocessing Data

In [19]:
# C4
# ########## From Oliver 2

# Size
target_size = (256, 256)

# List to store the preprocessed images
preprocessed_images = []

# List to store the preprocessed images
preprocessed_images = []

# Loop through the loaded images and preprocess them
for img in images:
    # Resize the image
    img_resized = img.resize(target_size)

    # Convert the image to grayscale (if we want to, unsure of the medical implications here)
    # img_resized = img_resized.convert("L")

    # Convert the image to a NumPy array and normalize pixel values
    img_array = np.array(img_resized) / 255.0

    # Append the preprocessed image to the list
    preprocessed_images.append(img_array)

# Convert the list to a NumPy array for easier handling
preprocessed_images = np.array(preprocessed_images)

# Pickle the preprocessed images
with open("preprocessed_images.pkl", "wb") as f:
    pickle.dump(preprocessed_images, f)

print("Preprocessing complete and data pickled.")

###Preprocessing complete and data pickled.

Preprocessing complete and data pickled.


In [20]:
# C5

# Display information about the image
print(img.format)
print(img.size)
print(img.mode)

# Show the image
img.show()

JPEG
(768, 768)
RGB


In [21]:
# C6

filenames_df = pd.DataFrame(preprocessed_images)
#filenames_df = pd.read_csv(path)
#filenames_df = pd.DataFrame(path)

#filenames_df.head()
#filenames_df.head()

,0


In [ ]:
# Print a random image from the list to ensure the import was successful
#images[40]

In [ ]:
# Check the size of the second image
#images[1].size

Part 3: Spliting Dataset

In [22]:
# C7

# Print the first few image filenames
filenames_df.head()

,0


In [ ]:
# C8

# First, remove the .png file extension, then split into two new columns to
# differentiate between lungaca, lungn, lungscc by fectching the 4th letter of
# the word of the image filename
#filenames_df[['userid', 'pose', 'expression', 'eyes']] = filenames_df['files']\
#                                                            .str.replace('.png', '', regex=False)\
#                                                            .str.split('_', expand=True)
filenames_df[['files', 'tumor_type']] = filenames_df['files']\
                                                            .str.replace('.jpeg', '', regex=False)\
                                                            .word.index[3], expand=True)
filenames_df.head()

In [ ]:
# C9

# Now we can call our preprocessed pixel data 'X'
X = normalized_images

# For our purposes, we'll select the userid column as 'y'
#y = filenames_df['userid']
y = filenames_df['tumor_type']

In [ ]:
# C10

# Check the total number of classes
y.nunique()

In [ ]:
# C11

# Convert values to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# C12

# Now we'll split our data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

Part 4: Augmentation

In [ ]:
# C13

# Apply augmentation to the whole training dataset
# Define the augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),         # Random rotation (20 degrees)
    tf.keras.layers.RandomTranslation(0.1, 0.1), # Random horizontal and vertical shift
    tf.keras.layers.RandomZoom(0.2),             # Random zoom
    tf.keras.layers.RandomFlip('horizontal')     # Random horizontal flip
])

# Create variables to hold the X and y training data
X_train_aug = []
y_train_aug = []

# Loop through all the images.
for i in range(len(X_train)):
    # Select the image
    img = X_train[i]
    # Select the label from the training data
    label = y_train[i]

    # Add a channel dimension for grayscale images
    img = np.expand_dims(img, axis=-1)  # Add channel dimension

    # Ensure that the input data has the correct shape
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Add 5 images for every original image
    for j in range(5):
        # Append a new image to the X list
        X_train_aug.append(data_augmentation(img, training=True)[0].numpy())
        # Append the label for the original image to the y list
        y_train_aug.append(label)

# Print the length of each list
print(len(X_train_aug))
print(len(y_train_aug))

In [ ]:
# C14

# Reshape test data for the model
X_test_np = []
for img in X_test:
    # Add a channel dimension for grayscale images
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    # Append the image to the list
    X_test_np.append(img)

# Convert to numpy array
X_test_np = np.array(X_test_np)

# Check the shape of the first image
X_test_np[0].shape

Part 5: Creating the Model

In [ ]:
# C15

# One hot encode the y data
y_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(np.array(y_train_aug).reshape(-1, 1))
y_train_aug_enc = y_encoder.transform(np.array(y_train_aug).reshape(-1, 1))
y_test_enc = y_encoder.transform(np.array(y_test).reshape(-1, 1))

# Convert values to numpy arrays
X_train_aug_np = np.array(X_train_aug)
X_test_np = np.array(X_test_np)
y_train_aug_np = np.array(y_train_aug_enc)
y_test_np = np.array(y_test_enc)

# Load and preprocess your CMU Face Images dataset (Ensure each image is labeled as "with sunglasses" or "without sunglasses")
# The following code assumes that you have already loaded and preprocessed your dataset into 'X' and 'y' (features and labels).

# Split the training dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_aug_np, y_train_aug_np, test_size=0.2, random_state=42)

# Print the total number of one_hot_encoded columns
np.array(y_train).shape

In [ ]:
# C16

# Define a CNN model
model = keras.Sequential([
    layers.Input((224, 224, 3)), #layers.Input((60, 64, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_train.shape[1], activation='sigmoid')  # the nubmer of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs
)

In [ ]:
# C17

# Evaluate the model using the testing data
model.evaluate(X_test_np, y_test_np)

## **Patient Interface with Gradio - Image**

In [ ]:
# C18

# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam".
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def lungtumor_prediction(image):
    """
    Predict the lung tumor classification of a given image using a pre-trained model.

    Parameters:
    - text (str): The image to be classified.

    Returns:
    - str: A message indicating whether the image is classified as benign or not.

    This function takes a lung tumor image and a pre-trained pipeline model, then predicts the
    tumor classification of the image. The result is a message stating whether the image is
    classified as cancerous or not.
    """
    # Create a variable that will hold the prediction of a new text.
    # Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
    text_clf = lungtumor_prediction(forilenames_df)

    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    print(f'The lungtumor: "{image}", is not cancerous.' if y = n, elif, f'The lungtumor: "{image}", is cancerous.', else, f'The lungtumor: "{image}", is cancerous.' if y = s )

In [ ]:
# C19

# Create a Gradio interface
!pip install gradio
# Import Gradio
import gradio as gr

In [ ]:
# C20

# Create an instance of the Gradio Interface application function with the appropriate parameters.
input_img = gr.inputs.Image(shape=(224, 224)) #input_img = gr.inputs.Image(shape=(150, 150))
output = 'text' #target_img = gr.outputs.Image()
app = gr.Interface(fn=text_clf, inputs=image, outputs=text, title='Is This Lung Tumor Cancerous?')
# Launch the app
app.launch()